In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import cv2

import numpy as np

Using TensorFlow backend.


In [2]:
row = 112
col = 112
channels = 3
latent_dim = 200
img_shape = (row, col, channels)
img_shape, latent_dim

((112, 112, 3), 200)

In [3]:
## DCNN : For upsampling
def build_generator():

    model = Sequential()

    model.add(Dense(512 * 28 * 28, activation="relu", input_dim=latent_dim))
    model.add(Reshape((28, 28, 512)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

build_generator()

def build_discriminator():

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)
build_discriminator()

W0711 15:02:32.387472 139661724616512 deprecation_wrapper.py:119] From /home/administrator123/anaconda3/envs/Image_Processing/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 15:02:32.435373 139661724616512 deprecation_wrapper.py:119] From /home/administrator123/anaconda3/envs/Image_Processing/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 15:02:32.443049 139661724616512 deprecation_wrapper.py:119] From /home/administrator123/anaconda3/envs/Image_Processing/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 15:02:32.484111 139661724616512 deprecation_wrapper.py:119] From /home/administrator123/anaconda3/envs/Image_Processing/lib/python3.6/site-package

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 401408)            80683008  
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 512)       0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 56, 56, 512)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 128)       589952    
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 56, 56, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 112, 112, 128)     0         
__________

In [4]:
optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the generator
generator = build_generator()

# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

print(img.shape, z)
# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 56, 56, 32)        896       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 56, 56, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 29, 29, 64)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 29, 29, 64)        256       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 29, 29, 64)        0         
__________

W0711 15:02:34.721603 139661724616512 deprecation_wrapper.py:119] From /home/administrator123/anaconda3/envs/Image_Processing/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0711 15:02:34.736473 139661724616512 deprecation.py:323] From /home/administrator123/anaconda3/envs/Image_Processing/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 401408)            80683008  
_________________________________________________________________
reshape_2 (Reshape)          (None, 28, 28, 512)       0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 56, 56, 512)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 56, 56, 128)       589952    
_________________________________________________________________
batch_normalization_9 (Batch (None, 56, 56, 128)       512       
_________________________________________________________________
activation_4 (Activation)    (None, 56, 56, 128)       0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 112, 112, 128)     0         
__________

In [8]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/pokemon_%d.png" % epoch)
    plt.close()

In [9]:
def train(epochs, batch_size=128, save_interval=500):

    
    X = np.zeros((112,112,3))
    
    instances = []

    # Load in the images
    for filepath in os.listdir('/home/administrator123/Akanksha/CNN/GAN/Pokemon/data/'):
#         instances.append(cv2.imread('/home/administrator123/Akanksha/CNN/GAN/Pokemon/data/'))       
        img = cv2.imread('/home/administrator123/Akanksha/CNN/GAN/Pokemon/data/' + filepath)
        img = cv2.resize(img, (112,112))
        instances.append(img)
              
    X_train = np.array(instances)
    print(X_train.shape)
    
    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
#     X_train = np.expand_dims(X_train, axis=3)
    
    print(X_train.shape)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = combined.train_on_batch(noise, valid)

        # Plot the progress
        if epoch % 1000 == 0 :
          print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [ ]:
import os
train(epochs=400000, batch_size=32, save_interval=500)

(100, 112, 112, 3)
(100, 112, 112, 3)


/home/administrator123/anaconda3/envs/Image_Processing/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.226326, acc.: 45.31%] [G loss: 1.722779]
